In [12]:
import pandas as pd
import jsonpickle
import numpy as np
import matplotlib.pyplot as plt
import os, os.path
import dateutil.parser
from datetime import datetime

Constants

In [13]:
FILE_AS_ROOT = False
USE_HISTORY = False
SELF_PATH = os.getcwd()#os.path.dirname(os.path.abspath(__file__))
PATH_TO_ROOT = os.path.join(SELF_PATH,'../../../../../../' if FILE_AS_ROOT else '')
PATH_TO_WRITE_CSV = os.path.join(PATH_TO_ROOT,'../')
PATH_TO_JSON_STATE = os.path.join(PATH_TO_ROOT,'../latest_state.json')

Given path to a pickle state,will return pickle object

In [14]:
def get_pickle(path_to_file):
    with open(path_to_file, 'r') as f:
        return jsonpickle.decode(f.read())

Given a pickgle object will return how many incomplete, complete images have been tagged along with unix epoch time and date time stamp

In [15]:
def how_many_tagged(pickle_file):
    
    done_tagging_count=len(pickle_file.finished_tagged_queue)
    tagged_but_not_done_count=0
    
    for image in pickle_file.pending_images_queue:
        if len(image.get_taggers()) > 0:
            tagged_but_not_done_count+=1
    
    return pd.DataFrame([{ 
        'not_done':tagged_but_not_done_count,
        'done':done_tagging_count,
        'tagged_ratio': tagged_but_not_done_count/(tagged_but_not_done_count+done_tagging_count)
     }])

In [16]:
pickle = get_pickle(PATH_TO_JSON_STATE)

In [17]:
# for image in pickle.finished_tagged_queue:
#     times_stop = image.stats_tagging_stop
#     keys = list(times_stop.keys())
#     last_tagger = keys[-1]
#     last_time = times_stop.get(last_tagger)
#     print(datetime.fromtimestamp(last_time))

In [18]:
finished_images = pickle.finished_tagged_queue

In [19]:
def gen_csv(history = True):
    image_tag_df = pd.DataFrame()

    for image in finished_images:

        tagger_list = list(image.get_taggers())
        image_id = image.get_rel_path()

        dict_row ={
            'image_id':image_id
            # 'tagging_start':image.stats_tagging_start,
            # 'tagging_end':image.stats_tagging_stop,
            # 'tagging_session_time':image.stats_tag_elapsed_session,
            # 'tagging_assingment_time':image.stats_tag_elapsed_assigned
        }
        
        # View all tags of an image, not final
        if history:
            for tagger in tagger_list:
                # Get all the tags tagged by this tagger
                list_of_tags = (image.get_tags(tagger))
                
                # Add those tags and the tagger ID to the dictionary that will be our row
                dict_row.update(list_of_tags)
                dict_row.update({
                    'tagger':tagger,
                    'tagging_start':image.stats_tagging_start[tagger],
                    'tagging_end':image.stats_tagging_stop[tagger],
                    'elapsed_session_time':image.stats_tag_elapsed_session[tagger],
                    'total_assingment_time':image.stats_tag_elapsed_assigned[tagger]
                })
                
                # Combine the dictionary row as a dataframe into the image tag dataframe
                tag_row_df = pd.DataFrame([dict_row]) 
                image_tag_df = pd.concat([image_tag_df,tag_row_df],sort=True , ignore_index = True)
        else:
            final_tags_df = image.final_tags 
            dict_row.update(final_tags_df)
            tag_row_df = pd.DataFrame([dict_row]) 
            #print(image.stats_tagging_start)
            image_tag_df = pd.concat([image_tag_df,tag_row_df],sort=True, ignore_index = True)
    return image_tag_df   

In [20]:
with_history_tags = gen_csv()
final_tags =gen_csv(False)

In [21]:
final_tags.to_csv(os.path.join(PATH_TO_WRITE_CSV,'image_final_tags.csv'))
with_history_tags.to_csv(os.path.join(PATH_TO_WRITE_CSV,'image_with_histoy_tags.csv'))

In [22]:
final_tags.head()

,development,image_id,impact,ocean,terrain_inland,terrain_marsh,terrain_river,terrain_sandy_coastline,terrain_undefined,washover
0,0.0,jpgs/S26047238.jpg,0.0,NaN,NaN,NaN,NaN,NaN,True,0.0
1,1.0,jpgs/P26057146.jpg,2.0,NaN,NaN,NaN,NaN,True,NaN,0.0
2,1.0,jpgs/P26054308.jpg,2.0,NaN,NaN,NaN,NaN,True,NaN,0.0
3,1.0,jpgs/P26050957.jpg,0.0,NaN,NaN,NaN,NaN,NaN,True,0.0
4,0.0,jpgs/P26057962.jpg,3.0,NaN,NaN,NaN,NaN,True,NaN,1.0
